In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
data_pre = pd.read_excel('./data.xlsx')

In [3]:
data_pre

,单位,总价,人力,方案,设备级别,公司级别,能力成熟度
0,A,220,6,30,10,10,5
1,B,190,8,25,9,8,3
2,C,180,8,28,7,7,4
3,D,170,7,23,8,7,2


In [4]:
label = data_pre.keys()[1:]

In [5]:
label

Index(['总价', '人力', '方案', '设备级别', '公司级别', '能力成熟度'], dtype='object')

In [6]:
value = data_pre[label].values.astype(np.float64)
data = copy.deepcopy(value) 

In [7]:
value

array([[220.,   6.,  30.,  10.,  10.,   5.],
       [190.,   8.,  25.,   9.,   8.,   3.],
       [180.,   8.,  28.,   7.,   7.,   4.],
       [170.,   7.,  23.,   8.,   7.,   2.]])

In [8]:
for i in range(len(label)):
    value[:, i] = value[:, i]/np.sqrt(np.sum(value[:, i] ** 2))

In [9]:
value

array([[0.57616108, 0.41111323, 0.56313829, 0.58321184, 0.61780206,
        0.68041382],
       [0.49759366, 0.54815097, 0.46928191, 0.52489066, 0.49424165,
        0.40824829],
       [0.47140452, 0.54815097, 0.52559574, 0.40824829, 0.43246144,
        0.54433105],
       [0.44521538, 0.4796321 , 0.43173936, 0.46656947, 0.43246144,
        0.27216553]])

In [10]:
# 获取权重
# 标准归一化
ymax = 1
ymin = 0.002
for i in range(len(label)):
    data[:, i] = (ymax - ymin) * (data[:, i] - np.min(data[:, i]))/(np.max(data[:, i]) - np.min(data[:, i])) + ymin

In [11]:
data

array([[1.        , 0.002     , 1.        , 1.        , 1.        ,
        1.        ],
       [0.4012    , 1.        , 0.28714286, 0.66733333, 0.33466667,
        0.33466667],
       [0.2016    , 1.        , 0.71485714, 0.002     , 0.002     ,
        0.66733333],
       [0.002     , 0.501     , 0.002     , 0.33466667, 0.002     ,
        0.002     ]])

In [12]:
# 计算信息熵
m = len(data_pre.index)
n = len(label)
P = np.zeros((m, n))
E = np.zeros(n)
for i in range(n):
    P[:, i] = data[:, i]/np.sum(data[:, i])

In [13]:
for i in range(n):
    E[i] = -1/np.log(m)*np.sum(P[:, i]*np.log(P[:, i]))

In [14]:
# 计算权重和得分
w1 = (1 - E)/np.sum(1 - E)

In [15]:
w1

array([0.1746776 , 0.11943681, 0.14179118, 0.13483358, 0.29442725,
       0.13483358])

In [16]:
R1 = value*w1

In [17]:
w2 = np.array([0.3724, 0.1003, 0.1991, 0.1991, 0.0998, 0.0485])
R2 = value*w2

In [18]:
R = R1
# R = R2

In [19]:
# 得到每个指标的距离
r_max = R.max(axis=0)
r_min = R.min(axis=0)

In [20]:
r_max

array([0.10064243, 0.0654694 , 0.07984804, 0.07863654, 0.18189776,
       0.09174263])

In [21]:
n = len(data_pre.index)
d_pos = np.array([])
d_neg = np.array([])
for i in range(n):
    # d+ 向量
    d_pos = np.append(d_pos, np.sqrt(np.sum((r_max - R1[i, :]) ** 2)))
    # d- 向量
    d_neg = np.append(d_neg, np.sqrt(np.sum((r_min - R1[i, :]) ** 2)))

In [22]:
score = d_neg/(d_pos + d_neg)

In [23]:
Score = 100 * score / np.max(score)

In [24]:
Score

array([100.        ,  46.72082004,  47.05845904,  14.04338538])

In [25]:
for i in range(n):
    print('单位{}得分为:{}'.format(data_pre['单位'][i], Score[i]))

单位A得分为:100.0
单位B得分为:46.720820042953235
单位C得分为:47.05845903818325
单位D得分为:14.04338537742974
